# 911 Calls Capstone Project

For this capstone project we will be analyzing some 911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)

Just go along with this notebook and try to complete the instructions or answer the questions in bold using your Python and Data Science skills!

## Data and Setup

____
** Import numpy and pandas **

In [ ]:
import numpy as np
import pandas as pd

** Import visualization libraries and set %matplotlib inline. **

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

** Read in the csv file as a dataframe called df **

In [ ]:
df = pd.read_csv('911.csv')

** Check the info() of the df **

In [ ]:
df.info()

** Check the head of df **

In [ ]:
df.head(3)

## Basic Questions

** What are the top 5 zipcodes for 911 calls? **

In [ ]:
df['zip'].value_counts().head(5)

** What are the top 5 townships (twp) for 911 calls? **

In [ ]:
df['twp'].value_counts().head(5)

** Take a look at the 'title' column, how many unique title codes are there? **

In [ ]:
df['title'].nunique()

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

In [ ]:
df['reason'] = df['title'].apply(lambda title: title.split(':')[0])
df.head(3)

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df['reason'].value_counts().head() # gets the most frequent occuring elements in this series, a column in a dataFrame is a series

** Now use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
import  seaborn as sns
sns.set_style('whitegrid')
sns.countplot(data=df, x='reason')
ax = plt.gca()
ax.set_ylim([0, 50000])

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
type( df['timeStamp'].iloc[0] ) # get the value in the 0 index of the TimeStamp Column

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df['timeStamp'] = df['timeStamp'].apply(pd.to_datetime)

In [ ]:
time = df['timeStamp'].iloc[0]
time.hour

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
df['hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['month'] = df['timeStamp'].apply(lambda time: time.month)
df['dayOfWeek'] = df['timeStamp'].apply(lambda time: time.day_of_week)
df.head(3)

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df['dayOfWeek'] = df['dayOfWeek'].map(dmap)
df.head(3)

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
sns.countplot(data=df, x='dayOfWeek', hue='reason')

ax = plt.gca()
ax.set_ylim([0, 8000])
ax.set_xlabel('Day of Week')

plt.legend(loc='upper left', bbox_to_anchor=(1.05,1.015))

**Now do the same for Month:**

In [ ]:
sns.countplot(data=df, x='month', hue='reason')

ax = plt.gca()
ax.set_ylim([0, 7000])
ax.set_xlabel('Month')

plt.legend(loc='upper left', bbox_to_anchor=(1.05,1.015))

**It is missing some months! 9,10, and 11 are not there.**

**Did you notice something strange about the Plot?**

_____

** You should have noticed it was missing some Months, let's see if we can maybe fill in this information by plotting the information in another way, possibly a simple line plot that fills in the missing months, in order to do this, we'll need to do some work with pandas... **

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
df.groupby('month').count().reset_index()

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
months_df = df.groupby('month').count()
months_df['dayOfWeek'].plot()
ax = plt.gca()
ax.set_xlim([1, 12])
ax.set_ylim([7000, 14000])
ax.set_xlabel('Month')

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
months_df = months_df.reset_index()
sns.lmplot(data=months_df, x='month', y='dayOfWeek')

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
df['Date'] = df['timeStamp'].apply(lambda time: time.date())
df.head(3)

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
byDate = df.groupby(by='Date').count()['lat'].plot(color='blue')

ax = plt.gca()
ax.set_ylim([100, 1000])
plt.tight_layout()

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
byDate = df[ df['reason'] == 'Traffic' ].groupby(by='Date').count()
byDate['lat'].plot(color='blue')

ax = plt.gca()
ax.set_ylim([0, 600])
ax.set_title('Traffic')
plt.tight_layout()

In [ ]:
byDate = df[df['reason'] == 'Fire'].groupby(by='Date').count()
byDate['lat'].plot(color='blue')

ax = plt.gca()
ax.set_ylim([0, 180])
ax.set_title('Fire')
plt.tight_layout()


In [ ]:
byDate = df[df['reason'] == 'EMS'].groupby(by='Date').count()
byDate['lat'].plot(color='blue')

ax = plt.gca()
ax.set_ylim([50, 250])
ax.set_title('EMS')
plt.tight_layout()

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
dayHour = df.groupby(by=['dayOfWeek', 'hour']).count()['reason'].unstack()
dayHour

** Now create a HeatMap using this new DataFrame. **

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(data=dayHour)
plt.tight_layout()

** Now create a clustermap using this DataFrame. **

In [ ]:
plt.figure(figsize=(12, 6))
sns.clustermap(data=dayHour)
plt.tight_layout()

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = df.groupby(by=['dayOfWeek', 'month']).count()['reason'].unstack()
dayMonth

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(data=dayMonth)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 6))
sns.clustermap(data=dayMonth)
plt.tight_layout()

**Continue exploring the Data however you see fit!**
# Great Job!

In [ ]:
df.head(3)

## Examining the Address That Made the Most 911 Calls

### Finding the Address That Made the Most 911 Calls

In [ ]:
df['addr'].value_counts()

### Graphing the 911 Calls From Shannondell Dr & Blvd During the Months

In [ ]:
shannondell = df[df['addr'] == 'SHANNONDELL DR & SHANNONDELL BLVD']
shannondellByDate = shannondell.groupby(by='month').count()['lat']

plt.figure(figsize=(12, 6))
shannondellByDate.plot.line(x='month')

axes = plt.gca()
axes.set_title('911 Calls From Shannondell Dr & Blvd During the Months')
axes.set_ylabel('Number of 911 Calls')
axes.set_xlabel('Month')
axes.set_xlim([1, 12])
axes.set_ylim([70, 140])
plt.tight_layout()

#### Examining the Months with the Most Calls

In [ ]:
sdTopFiveMonths = shannondell['month'].value_counts().head(5).index
sdTopFiveMonths

#### What Type of 911 Call were the biggest in the Top 3 Months

In [ ]:
sdJanuary = shannondell[shannondell['month'] == sdTopFiveMonths[0]]
sdJanuary['reason'].value_counts()

In [ ]:
sdJuly = shannondell[shannondell['month'] == sdTopFiveMonths[1]]
sdJuly['reason'].value_counts()

In [ ]:
sdApril = shannondell[shannondell['month'] == sdTopFiveMonths[2]]
sdApril['reason'].value_counts()

##### What type of EMS were the Most Common in the Top 3 Months

In [ ]:
sdJanuaryReasons = sdJanuary[ sdJanuary['reason'] == 'EMS']['title'].value_counts()

plt.figure(figsize=(15, 10))
sdJanuaryReasons.plot.pie(autopct='%1.1f%%')

axes = plt.gca()
axes.set_title('Most Common Types 911 Calls From Shannondell Dr & Blvd During the January')
axes.set_ylabel('')
plt.tight_layout()

#### Is there a Correaltion between EMS and Date in Shannondell

Creating the pivot table

In [ ]:
pvSdMonthTitle = shannondell.groupby(by=['month', 'title']).count()['lat'].unstack()
pvSdMonthTitle

In [ ]:
pvSdMonthTitle.fillna(value=0, inplace=True)

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(data=pvSdMonthTitle, annot=True)

axes = plt.gca()
axes.set_title('Months EMS Appear In')
axes.set_ylabel('Month')
axes.set_xlabel('Title')
plt.tight_layout()

##### Zooming in on the top 3 EMS reasons

In [ ]:
sdTop3Reasons = shannondell[(shannondell['title'] == 'EMS: CARDIAC EMERGENCY') |
            (shannondell['title'] == 'EMS: FALL VICTIM') | 
            (shannondell['title'] == 'EMS: MEDICAL ALERT ALARM')]

pvsdTop3ReasonsMonthTitle = sdTop3Reasons.groupby(
    by=['month', 'title']).count()['lat'].unstack()
pvsdTop3ReasonsMonthTitle

plt.figure(figsize=(6, 6))
sns.heatmap(data=pvsdTop3ReasonsMonthTitle, annot=True)

axes = plt.gca()
axes.set_title('Months EMS Appear In')
axes.set_ylabel('Month')
axes.set_xlabel('Title')
plt.tight_layout()